In [1]:
!pip install transformers
!pip install sentence_transformers
!pip install datasets
!pip install accelerate
!pip install sentencepiece


^C



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from transformers import AutoModelForSeq2SeqLM ,TrainingArguments, AutoTokenizer , DataCollatorForSeq2Seq,Seq2SeqTrainer , Trainer , MarianMTModel, MarianTokenizer
from datasets import Dataset , load_dataset
import tensorflow as tf
import json
import ast

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


c:\Users\7fno\AppData\Local\Programs\Python\Python311\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [5]:
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ar-en")

In [6]:
data_collator = DataCollatorForSeq2Seq(tokenizer , model = model,return_tensors=  'tf')

In [19]:
data = pd.read_csv(r'D:\تالتة اعدادي دراسات\Book12.csv')

In [20]:
data

,عربي,English
0,شهد العالم الاسلامي أواخر القرن الخامس عشر و ب...,The Islamic world witnessed an intense conflic...
1,دولة المماليك قامت في مصر و الشام و الحجاز و ج...,"The Mamluk state was established in Egypt, the..."
2,وتحطم الاسطول المملوكي عام 1509م في معركة ديو ...,", and the destruction of the Mamluk fleet in 1..."
3,الدولة العثمانية مع بداية القرن السادس عشر الم...,The Ottoman Empire: At the beginning of the si...
4,الدولة الصفوية و بعد ان تمكن السلطان سليم الاو...,The Safavid state. After Sultan Selim I was ab...
...,...,...
221,اجتمعت عوامل كثيرة ساهمت في اندلاع ثورة 1919م،...,Many factors came together that contributed to...
222,زيادة نمو الوعي الوطني لدى المصريين من أجل الا...,Increased growth in national awareness among E...
223,الإجراءات العنيفة التي اتخذتها سلطات الاحتلال ...,Violent measures taken by the occupation autho...
224,تدعي إنجلترا وحلفاؤها أنهم يقاتلون من أجل حرية...,England and its allies claim that they are fig...


In [21]:
tokenizer('اهلا انا محمد')

{'input_ids': [17037, 30, 842, 7434, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [22]:
data = Dataset.from_pandas(data)
data = data.train_test_split(.1)

In [23]:
data

DatasetDict({
    train: Dataset({
        features: ['عربي', 'English'],
        num_rows: 203
    })
    test: Dataset({
        features: ['عربي', 'English'],
        num_rows: 23
    })
})

In [24]:
data['train'][:1]

{'عربي': ['الحملة الفرنسية على الشام 1799م:الأسباب: تحالفت الدولة العثمانية مع إنجلترا وروسيا لطرد الفرنسيين من مصر بالقوة العسكرية، فأعدوا حملتين لغزو مصر، الأولى بحرية على جزيرة رودس إلى مصر من الشمال، والثانية برية إلى مصر من الشرق عبر بلاد الشام.'],
 'English': ['The French campaign against the Levant, 1799 AD: Reasons: The Ottoman Empire allied with England and Russia to expel the French from Egypt by military force, so they prepared two campaigns to invade Egypt, the first freely on the island of Rhodes to Egypt from the north, and the second overland to Egypt from the east via the Levant.']}

In [25]:
data['train'][0]

{'عربي': 'الحملة الفرنسية على الشام 1799م:الأسباب: تحالفت الدولة العثمانية مع إنجلترا وروسيا لطرد الفرنسيين من مصر بالقوة العسكرية، فأعدوا حملتين لغزو مصر، الأولى بحرية على جزيرة رودس إلى مصر من الشمال، والثانية برية إلى مصر من الشرق عبر بلاد الشام.',
 'English': 'The French campaign against the Levant, 1799 AD: Reasons: The Ottoman Empire allied with England and Russia to expel the French from Egypt by military force, so they prepared two campaigns to invade Egypt, the first freely on the island of Rhodes to Egypt from the north, and the second overland to Egypt from the east via the Levant.'}

In [26]:
len(data['train'])

203

In [27]:
data

DatasetDict({
    train: Dataset({
        features: ['عربي', 'English'],
        num_rows: 203
    })
    test: Dataset({
        features: ['عربي', 'English'],
        num_rows: 23
    })
})

In [32]:
max_input =256
max_output= 256
def process(example):
  # print(example['عربي'][0])
  inputs = [ss for ss in example['عربي']]
  target = [ss for ss in example['English']]
  model_inputs = tokenizer(inputs , max_length=256 , truncation=True)
  with tokenizer.as_target_tokenizer():
   labels = tokenizer(target , max_length=max_output , truncation=True)
  model_inputs['labels'] = labels ['input_ids']
  return model_inputs

In [33]:
tokenized_data = data.map(process, batched=True,remove_columns=data['train'].column_names)

Map:   0%|          | 0/203 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

In [34]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 203
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 23
    })
})

In [ ]:
tokenizer.decode(tokenized_data['train']['input_ids'][3])

'اتجه العثمانيين الي الي مصر و الشام بسبب رغبة السلطان سليم الاول في توسيع ممتلكاته في الشرق عن طريق الاستيلاء علي مصر التي تمثل قلب العالم الاسلامي و ايضا بسبب ادراك السلطان سليم الاول ان الصدام مع الدولة الاوروبية</s>'

In [ ]:
# process(data['train'])

In [ ]:
# dataes = []
# for i in range(17):
#   answer_str = data['train']['Translation'][i]
#   datas = json.loads(answer_str.replace("'", "\""))
#   dataes.append(datas)

In [ ]:
# dataes[0]['en']

In [35]:
data_collator = DataCollatorForSeq2Seq(model = model  ,tokenizer = tokenizer)

In [36]:
tokenized_data['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 203
})

In [ ]:
training_args = TrainingArguments('finetuned_model',learning_rate = 2e-5, per_device_train_batch_size=32,
                                      num_train_epochs  = 20 ,
                                      evaluation_strategy = 'epoch')

In [ ]:
trainer = Trainer(model=model, args=training_args
                         ,train_dataset = tokenized_data['train']
                         ,eval_dataset = tokenized_data['test']
                         ,data_collator = data_collator
                         ,tokenizer = tokenizer)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.610756
2,No log,1.527933
3,No log,1.443931
4,No log,1.374197
5,No log,1.319512
6,No log,1.275993
7,No log,1.238244
8,No log,1.205847
9,No log,1.178291
10,No log,1.154274


TrainOutput(global_step=20, training_loss=0.85272216796875, metrics={'train_runtime': 521.5018, 'train_samples_per_second': 0.652, 'train_steps_per_second': 0.038, 'total_flos': 8193870397440.0, 'train_loss': 0.85272216796875, 'epoch': 20.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/trained_models/translation_model.h5')

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('my_translation_model')
tokenizer = AutoModelForSeq2SeqLM.from_pretrained("my_translation_model")



In [ ]:
model_name = 'my_translation_model'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

input_text = '''لماذا عانت الدولة الصفوية'''

# Tokenize input text
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate translations
output = model.generate(input_ids)

# Decode the generated output
translated_text = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
print(translated_text)

Why did the Safavid state suffer?


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save_pretrained('/content/drive/MyDrive/trained_models')